# Exercices SQL

Récupérer les données dans 'produits csv'

Les insérer dans une nouvelle base de deonnées qu’on aura conçue
Faire ça en python



In [109]:
import numpy as np
import pandas as pd

prod_df = pd.read_csv('./Produits - Feuille 1.csv')

print(prod_df.shape[0])

fam_df = prod_df.groupby(['FAMILLE ARTICLE']).count().drop(['LIBELLE ARTICLE', 'CONDITION', 'PU HT'], axis=1)
cond_df = prod_df.groupby(['CONDITION']).count().drop(['LIBELLE ARTICLE', 'FAMILLE ARTICLE', 'PU HT'], axis=1)
# print(fam_df.sort_values('Code article', ascending=False))
print(cond_df.sort_values('Code article', ascending=False))

1863
           Code article
CONDITION              
BOITE               395
CARTO               188
SACHE               162
SAC                 131
BOUTE               112
PAQUE               110
KG                   99
POT                  79
FARDE                71
SEAU                 57
BIDON                54
PIECE                40
BARQU                40
PLATE                39
UNI                  39
FLACO                38
SQUEE                36
LOT                  29
ETUI                 29
PACK                 26
BOCAL                26
BRICK                18
TUBO                  8
POCHE                 7
CAISS                 7
SPRAY                 7
TUBE                  4
BAC                   4
PINGO                 3
BIB                   3
COLIS                 1
BRK                   1


Une fois qu’on a ces infos, on sait qu’on devra créer une table pour chacunes de ces informations en y associant un identifiant

Se connecter avec PyMysql

In [7]:
import mysql.connector
import random

db = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'root',
    database = 'sakila'
)

cur = db.cursor()

cur.execute("SELECT COUNT(*) FROM actor")
res = cur.fetchall()
print(res)


[(200,)]


Créons la Base de Données sur mon server

In [8]:
#create random id
actor_ids = list(range(1,201))
random.shuffle(actor_ids)
print(actor_ids[0])

sql = '''
SELECT f.title, a.first_name, a.last_name, a.actor_id
FROM film_actor AS fa
JOIN film AS f
ON fa.film_id = f.film_id
JOIN actor AS a
ON fa.actor_id = a.actor_id
WHERE fa.actor_id = %s;
'''

cur.execute(sql, (actor_ids[0],))
res = cur.fetchall()
print(len(res))


34
25


Essayons de créer une table sur ma base perso «produ1»

In [102]:
db = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'root',
    database = 'produ1'
)

cur = db.cursor()

cur.execute("DROP TABLE produit")
cur.execute("DROP TABLE famille")
cur.execute("DROP TABLE boudin")

req = ['''
CREATE TABLE IF NOT EXISTS produit (
    prod_id int,
    fam_id int,
    cond_id int,
    libelle varchar(80) NOT NULL UNIQUE,
    prix decimal(6,2) NOT NULL,
    PRIMARY KEY (prod_id)
)''','''
CREATE TABLE IF NOT EXISTS famille (
    fam_id int,
    nom varchar(80) NOT NULL UNIQUE,
    PRIMARY KEY (fam_id)
)''','''
CREATE TABLE IF NOT EXISTS condit (
    cond_id int,
    nom varchar(80) NOT NULL UNIQUE,
    PRIMARY KEY (cond_id)
)
''']

for r in req : cur.execute(r)

cur.execute("SHOW TABLES")

for x in cur: print(x)

('condit',)
('famille',)
('produit',)


In [107]:

# INSERTS INTO FAMILLE
# purge de la table
cur.execute('TRUNCATE TABLE famille')

# préparer les données
fam_df = fam_df.sort_values('Code article', ascending=False)
famindex = fam_df.reset_index()
famindex['Code article'] = range(1,31)
famindex.columns = ('nom', 'fam_id')

# création d’une table des inserts
inserts = []
for i, r in famindex.iterrows():
    inserts.append(f"({r['fam_id']}, '{r['nom']}')")

# création du megastring de la requête
sql = f'''
INSERT INTO famille (fam_id, nom)
VALUES
{', '.join(inserts)}
'''

cur.execute(sql)
db.commit()



In [108]:
# INSERTS INTO CONDITIONNEMENTS
# purge de la table
cur.execute('TRUNCATE TABLE condit')

# préparer les données
cond_df = cond_df.sort_values('Code article', ascending=False)
condindex = cond_df.reset_index()
condindex['Code article'] = range(1,33)
condindex.columns = ('nom', 'cond_id')

# création d’une table des inserts
inserts = []
for i, r in condindex.iterrows():
    inserts.append(f"({r['cond_id']}, '{r['nom']}')")

# création du megastring de la requête
sql = f'''
INSERT INTO condit (cond_id, nom)
VALUES
{', '.join(inserts)}
'''

cur.execute(sql)
db.commit()

In [136]:
# remplir la table principale

fam_dic = famindex.to_dict('tight')
cond_dic = condindex.to_dict('tight')
repl_dict = {
    "FAMILLE ARTICLE": dict(fam_dic['data']),
    "CONDITION": dict(cond_dic['data'])
}

main_df = prod_df.copy()

for k,v in repl_dict.items() : {
    main_df[k].replace(v, inplace = True)
}

print(main_df)

      Code article                           LIBELLE ARTICLE  FAMILLE ARTICLE  \
0             1010                       HUILE ARACHIDE 25 L               17   
1             1011                        HUILE ARACHIDE 5 L               17   
2             1012                        HUILE ARACHIDE 1 L               17   
3             1014    HUILE MEZZO 1 L (50%OLIVE50%TOURNESOL)               17   
4             1015                           HUILE COLZA 5 L               17   
...            ...                                       ...              ...   
1858         35885                 QUICHES LORRAINE 180G X14                1   
1859         35900                FARCIS NICOIS SG - SAC 2KG                1   
1860         35920              RAVIOLIS A LA NICOISE SG 2KG                1   
1861         37718               COOKIES CHOCOLAT SG 76G X30                1   
1862         37720  COOKIES 3 CHOC LAIT EMB.INDIV.106GX30 SG                1   

      CONDITION  PU HT  
0 